In [1]:
# Fist install the library

#%pip install rapidae

Since Rapidae uses the new version of Keras 3, this allows the use of different backends. 
We can select among the 3 available backends (Tensorflow, Pytorch and Jax) by modifying the environment variable "KERAS_BACKEND".
In the next cell we can define it.

In [2]:
import os

os.environ["KERAS_BACKEND"] = "torch"

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras import utils
from rapidae.data.datasets import load_MNIST
from rapidae.data.utils import evaluate
from rapidae.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from rapidae.models.vae.vae_model import VAE
from rapidae.pipelines.training import TrainingPipeline

ModuleNotFoundError: No module named 'tensorflow'

Download and preprocess the dataset. In this example, the selected dataset is the well-known MNIST composed of handwritten number images.

The "persistant" parameter of the load_MNIST() serves as a flag to determine if we want the dataset to be cached in the datasets folder.

In [4]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255

# Obtaint number of clasess
n_classes = len(set(y_train))


2023-12-22 01:20:00 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-22 01:20:00 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-22 01:20:00 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-22 01:20:00 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


We also need to convert the labels into one-hot encoding.

In [ ]:
# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

In this example we are using a variational autoencoder with a classifier attached to its latent space. Weights have been included for the classifier loss as well as for the vae loss in order to increase the classifying capability of the model.

In [6]:
# Model creation
model = VAE(input_dim=(x_train.shape[1], x_train.shape[2]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes,
            weight_vae=0.25, weight_clf=20)

2023-12-22 01:20:06 [INFO]: Classificator available for the latent space of the autoencoder


In [6]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=6, batch_size=128)

trained_model = pipe(x=x_train.astype(float), y=y_train)

2023-12-20 19:45:06 [INFO]: +++ training_pipeline +++
2023-12-20 19:45:06 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-20_19-45-06


Epoch 1/6


/home/lucaspc/venvs/keras_core/lib/python3.11/site-packages/keras/src/backend/common/backend_utils.py:88: UserWarning: You might experience inconsistencies accross backends when calling conv transpose with kernel_size=3, stride=2, dilation_rate=1, padding=same, output_padding=1.
  warnings.warn(



Epoch 1: loss improved from inf to 1390968438784.00000, saving model to ../output_dir/training_pipeline_2023-12-20_19-45-06/model.weights.h5
469/469 - 5s - 10ms/step - clf_loss: 6.9610 - kl_loss: 5563873755136.0000 - loss: 1390968438784.0000 - reconstruction_loss: 7217.3184
Epoch 2/6

Epoch 2: loss improved from 1390968438784.00000 to 79950288.00000, saving model to ../output_dir/training_pipeline_2023-12-20_19-45-06/model.weights.h5
469/469 - 4s - 9ms/step - clf_loss: 5.1702 - kl_loss: 319793824.0000 - loss: 79950288.0000 - reconstruction_loss: 7217.2090
Epoch 3/6

Epoch 3: loss improved from 79950288.00000 to 47203956.00000, saving model to ../output_dir/training_pipeline_2023-12-20_19-45-06/model.weights.h5
469/469 - 4s - 9ms/step - clf_loss: 4.9912 - kl_loss: 188808096.0000 - loss: 47203956.0000 - reconstruction_loss: 7217.2090
Epoch 4/6

Epoch 4: loss improved from 47203956.00000 to 29403152.00000, saving model to ../output_dir/training_pipeline_2023-12-20_19-45-06/model.weights.

In [7]:
model.load_weights(filepath='../output_dir/training_pipeline_2023-11-25_00-50-30/model.weights.h5')

Let's now display the label clusters of the latent space.

In [8]:
def plot_label_clusters(vae, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = vae.encoder.predict(data, verbose=0)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

SyntaxError: invalid syntax (87367603.py, line 1)

In [ ]:
plot_label_clusters(model, x_train, y_train)
plot_label_clusters(model, x_test, y_test)

Let's now check the classification metrics.

In [ ]:
from sklearn.metrics import classification_report

#y_train_predict = model.predict(train_data)
y_test_predict =  model.predict(x_train)

print(np.argmax(y_test_predict['clf'], axis=1))
print(np.argmax(y_test, axis=1))

target_names = ['number 0', 'number 1', 'number 2', 'number 3', 'number 4', 'number 5', 'number 6', 'number 7', 'number 8', 'number 9']

#classification_report(y_train, np.argmax(y_train_predict['clf'], axis=1), target_names=target_names)
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_test_predict['clf'], axis=1), target_names=target_names))

In [ ]:
y_hat = trained_model.predict(x_test)

evaluate(y_true=np.argmax(y_test, axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)